In [5]:
from roboflow import Roboflow
import open3d as o3d
import numpy as np
import math
import pyransac3d as pyrsc
import cv2
from PIL import Image

In [6]:
show_pcl = True

In [7]:
def visualizer(cloud):

    if type(cloud) != list:
        cloud = [cloud]

    if show_pcl:
        # point cloud
        center_of_mass = np.mean(np.asarray(cloud[0].points), axis=0)
    
        o3d.visualization.draw_geometries(cloud,
                                    zoom=0.57899999999999985,
                                    front=  [ 0.053781796277127751, 0.99511863815317547, 0.082743062484869914 ],
                                    up =  [ -0.99452345262671604, 0.045944145215534776, 0.093873519673282182 ],
                                    lookat=[ 0.14950467828195535, -0.21448131248991498, 0.63221199653621662 ])
    
    else:
        pass

In [10]:
pcd = o3d.io.read_point_cloud("../3dmodels/Bell_Pepper_6_26.ply")

visualizer(pcd)

In [44]:
o3d.visualization.draw_geometries_with_editing([pcd])

[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #0 (0.83, -0.14, 1.8) to add in queue.
[Open3D INFO] Picked point #0 (0.83, -0.14, 1.8) to add in queue.


In [41]:
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector([[0.98351237, 0.99766193, 0.04010914]])
point_cloud.paint_uniform_color((255, 0, 0))
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=pcd_new.get_center())

new_pcd_2 = point_cloud+pcd_new
visualizer([new_pcd_2])

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


In [31]:
pcd_new = o3d.io.read_point_cloud("../3dmodels/Bell_Pepper_6_26.ply")
visualizer(pcd_new)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [42]:
o3d.io.write_point_cloud(f"new_test_pcd.ply", new_pcd_2)


True

In [24]:
pcd.points

std::vector<Eigen::Vector3d> with 604420 elements.
Use numpy.asarray() to access data.

In [12]:
import open3d as o3d
import numpy as np

# Load point cloud (replace this with your own point cloud loading code)

# Define rotation angles for capturing different views
angles = np.linspace(0, 360, 36, endpoint=False)  # 36 frames for 360-degree rotation

# Create a renderer object
vis = o3d.visualization.Visualizer()
vis.create_window()

# Add point cloud to the visualizer
vis.add_geometry(pcd)

# Capture frames
images = []
for angle in angles:
    ctr = vis.get_view_control()
    ctr.rotate(angle, 0.0)  # Rotate around the vertical axis
    vis.poll_events()
    vis.update_renderer()
    
    # Capture the current visualization as an image
    image = vis.capture_screen_float_buffer(do_render=True)
    images.append(np.asarray(image)*255)  # Convert to uint8 for imageio compatibility

vis.destroy_window()


In [ ]:
# import imageio
# imageio.mimsave('point_cloud.gif', images, duration=0.1)  # Adjust duration as needed (in seconds)

In [733]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=pcd.get_center())

visualizer([pcd, mesh_frame])

In [720]:
# -Z
# np.asarray(pcd.points)[:, 2] = -np.asarray(pcd.points)[:, 2]

In [732]:
visualizer(pcd)

In [107]:
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

In [6]:
x_shape, z_shape = 3000, 3500
resolution = 11

In [144]:
# x__length, z_length
check_grid = np.zeros((x_shape, z_shape)) #They will set to 1 if that [x, y] grid is used; 0 -> false
grid = np.zeros((x_shape, z_shape)) #the actual grid which will have indexes mapped to a map
img = np.ones((x_shape, z_shape, 3), dtype=np.uint8) * 255
data = {} #{-1:[('RGB'), ('XYZ')], } #the data. key is index number saved into grid and values are the RGB and xyz data
# Along Y-Axis
points_sort = points[np.argsort(points[:, 1])[::-1]]
colors_sort = colors[np.argsort(points[:, 1])[::-1]]

max_val = np.max(np.asarray(points_sort), axis=0)
min_val = np.min(np.asarray(points_sort), axis=0)

cent_x = (min_val[0]+max_val[0])/2
range_x = np.abs(min_val[0]) + np.abs(max_val[0])
x_r_new = x_shape - 1

cent_z = (min_val[2]+max_val[2])/2
range_z = np.abs(min_val[2]) + np.abs(max_val[2])
z_r_new = z_shape - 1

for idx, val in enumerate(points_sort):
    x_c = int(((val[0] - cent_x) / range_x) * x_r_new + x_r_new/2)
    z_c = int(((val[2] - cent_z) / range_z) * z_r_new + z_r_new/2)

    if not check_grid[x_c, z_c]:

        check_grid[x_c, z_c] = 1
        data[idx] = [colors_sort[idx], val]
        grid[x_c, z_c] = idx
        col = np.uint8(colors_sort[idx]*255)
        img[x_c, z_c] = col
        # do around the point too

        for dx in range(-resolution, resolution, 1):
            for dz in range(-resolution, resolution, 1):

                if 0 <= x_c + dx < x_shape and 0 <= z_c + dz < z_shape:
                    if not check_grid[x_c + dx, z_c + dz]:
                        check_grid[x_c + dx, z_c + dz] = 1
                        img[x_c + dx, z_c + dz] = col

# 15 sec

image = Image.fromarray(img)
img_path = "numpy/numpy_image_31.jpg"
image.save(img_path)

In [118]:
# # About +Z axis

# x_shape, y_shape = 3000, 3500
# resolution = 11

# check_grid_2 = np.zeros((x_shape, y_shape)) #They will set to 1 if that [x, y] grid is used; 0 -> false
# grid_2 = -np.zeros((x_shape, y_shape)) #the actual grid which will have indexes mapped to a map
# img_2 = np.ones((x_shape, y_shape, 3), dtype=np.uint8) *255
# data_2 = {} #{-1:[('RGB'), ('XYZ')], } #the data. key is index number saved into grid and values are the RGB and xyz data


# # jUST CHANGING HERE
# points_sort_2 = points[np.argsort(points[:, 2])]
# colors_sort_2 = colors[np.argsort(points[:, 2])]
# # jUST CHANGING HERE

# point_cloud = o3d.geometry.PointCloud()
# point_cloud.points = o3d.utility.Vector3dVector(points_sort_2)
# point_cloud.colors = o3d.utility.Vector3dVector(colors_sort_2)
# mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=point_cloud.get_center())
# visualizer([point_cloud, mesh_frame])
# # break

# max_val_2 = np.max(np.asarray(points_sort_2), axis=0)
# min_val_2 = np.min(np.asarray(points_sort_2), axis=0)

# cent_x_2 = (min_val_2[0]+max_val_2[0])/2
# range_x_2 = np.abs(min_val_2[0]) + np.abs(max_val_2[0])
# x_r_new_2 = x_shape - 1

# cent_y_2 = (min_val_2[1]+max_val_2[1])/2
# range_y_2 = np.abs(min_val_2[1]) + np.abs(max_val_2[1])
# y_r_new_2 = y_shape - 1

# for idx, val in enumerate(points_sort_2):
#     x_c = int(((val[0] - cent_x_2) / range_x_2) * x_r_new_2 + x_r_new_2/2)
#     y_c = int(((val[1] - cent_y_2) / range_y_2) * y_r_new_2 + y_r_new_2/2)

#     if not check_grid_2[x_c, y_c]:
#         check_grid_2[x_c, y_c] = 1
#         data_2[idx] = [colors_sort_2[idx], val]
#         grid_2[x_c, y_c] = idx
#         col = np.uint8(colors_sort_2[idx]*255)
#         img_2[x_c, y_c] = col
        
#         # do around the point too
#         for dx in range(-resolution, resolution, 1):
#             for dz in range(-resolution, resolution, 1):
#                 if 0 <= x_c + dx < x_shape and 0 <= y_c + dz < z_shape:
#                     if not check_grid_2[x_c + dx, y_c + dz]:
#                         check_grid_2[x_c + dx, y_c + dz] = 1
#                         img_2[x_c + dx, y_c + dz] = col

# # 15 sec

In [145]:
rf = Roboflow(api_key="p0DKMSH5Ym3FH9zh7ZLa")
project = rf.workspace().project("sweet-pepper-detection")
model = project.version(1).model
predection_data = model.predict(img_path, confidence=80, overlap=30).json()

# visualize your prediction
model.predict(img_path, confidence=80, overlap=30).save(f"predictions_new/{img_path}")

loading Roboflow workspace...
loading Roboflow project...


In [146]:
predection_data

{'predictions': [{'x': 1473,
   'y': 1061,
   'width': 508,
   'height': 623,
   'confidence': 0.9209105372428894,
   'class': '49',
   'class_id': 0,
   'detection_id': '0e42e9a6-8235-444a-b511-1f475bc1e793',
   'image_path': 'numpy/numpy_image_31.jpg',
   'prediction_type': 'ObjectDetectionModel'}],
 'image': {'width': '3500', 'height': '3000'}}

In [147]:
# 2D bounding box coordinates
bounding_box = {
    'z': predection_data['predictions'][0]['x'],
    'x': predection_data['predictions'][0]['y'],
    'width': predection_data['predictions'][0]['width'],
    'height': predection_data['predictions'][0]['height'],
}

# Image dimensions (to scale 2D to 3D)
# image_width = predection_data['image']['width']
# image_height = predection_data['image']['height']

# Calculate 3D coordinates (assuming z-coordinate is arbitrary)
x_min = int(bounding_box['x'] - bounding_box['height']/2)
z_min = int(bounding_box['z'] - bounding_box['width']/2)
x_max = int(bounding_box['x']  + bounding_box['height']/2)
z_max = int(bounding_box['z'] + bounding_box['width']/2)

In [148]:
x_min, x_max, z_min, z_max

(749, 1372, 1219, 1727)

In [167]:
selected_colors = []
selected_points = []
# img_test = cv2.imread('predictions\prediction22.jpg')

for x in range(x_min, x_max, 1):
    for z in range(z_min, z_max, 1):
        idx = grid[x, z]
        selected_colors.append(data[idx][0])
        selected_points.append(data[idx][1])
        # img_test[x, z] = [255, 0, 0] #BRG

# cv2.imwrite("testimg.jpg", img_test)
# print(selected)

In [168]:
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(selected_points)
point_cloud.paint_uniform_color((255, 0, 0))

mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=pcd.get_center())

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


In [169]:
visualizer([point_cloud, pcd, mesh_frame])

In [161]:
max_val = np.max(np.asarray(pcd.points), axis=0)[1]
min_val = np.min(np.asarray(pcd.points), axis=0)[1]

max_x = np.max(selected_points, axis=0)
min_x = np.min(selected_points, axis=0)

print(max_x)

np.array([max_x, min_x])
# if along x-axis, i want all the x_axis values

# for i in range(min_val, max_val):
#     pass

[0.98351237 0.99766193 0.88431971]


array([[ 0.98351237,  0.99766193,  0.88431971],
       [-0.01939701, -0.98214686,  0.04010914]])

In [165]:
two = o3d.geometry.PointCloud()
two.points = o3d.utility.Vector3dVector(np.array([max_x, min_x]))
two.paint_uniform_color((255, 0, 0))
o3d.visualization.draw_geometries([pcd, two])

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


In [170]:
all = o3d.geometry.PointCloud()
all.points = o3d.utility.Vector3dVector(np.array(selected_points))
all.paint_uniform_color((255, 0, 0))
o3d.visualization.draw_geometries([pcd, all])

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


In [173]:
# Extract selected points
selected_points_cloud = point_cloud

# Compute axis-aligned bounding box (AABB)
bbox = point_cloud.get_oriented_bounding_box()

# Visualize point cloud and bounding box
o3d.visualization.draw_geometries([pcd, bbox])

# Get all points within the bounding box
pcd_crop = pcd.crop(bbox)

# Visualize points within bounding box
o3d.visualization.draw_geometries([pcd_crop])

In [3]:
def read_pcd(path):
    pcd = o3d.io.read_point_cloud(path)

    visualizer(pcd)

    return pcd

In [74]:
# paths = ['pcd_0.ply', 'pcd_1.ply', 'pcd_2.ply', 'pcd_3.ply', 'pcd_4.ply', 'pcd_5.ply', 'pcd_6.ply']
paths = ["../best bell pepper results/pcd_2.ply", "../best bell pepper results/pcd_4.ply", "../best bell pepper results/pcd_5.ply" ]
# paths = ["final_pcd_bell_pepper.ply"]
all_pcd = []
new_pcd = pcd
for path in paths:
    temp_pcd = read_pcd(path)

    bbox = temp_pcd.get_axis_aligned_bounding_box()

    visualizer([pcd, bbox])

    new_pcd = new_pcd.crop(bbox)

    visualizer(new_pcd)

    all_pcd.append(np.asarray(temp_pcd.points))

visualizer(new_pcd)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [85]:
def draw_one_bounding_box(pcd, all_points):

    # all_point_cloud = o3d.geometry.PointCloud()

    # for i in all_points:
    #     all_point_cloud += i

    bbox = all_points.get_axis_aligned_bounding_box()

    # Get the 8 corners of the bounding box
    corners = np.asarray(bbox.get_box_points())

    # right face data[0], data[1], data[2], data[7]

    # top right = 0, 2
    # bottom right = 1, 7
    # top left = 3, 5
    # bottom left = 4, 6

    # Define the edges of the bounding box
    edges = [
        [1, 7], [7, 4], [4, 6], [6, 1],  # Bottom rectangle
        [0, 2], [2, 5], [5, 3], [3, 0],  # Top rectangle
        [1, 0], [7, 2], [4, 5], [6, 3]   # Connections between top and bottom
    ]

    # Create a LineSet geometry to visualize edges
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(corners)
    line_set.lines = o3d.utility.Vector2iVector(edges)

    # Assign color to the LineSet (red color)
    colors = [[1, 0, 1] for _ in range(len(edges))]
    line_set.colors = o3d.utility.Vector3dVector(colors)    # for cld in all_points:

    #     # bbox = cld.get_oriented_bounding_box()
    #     bbox = cld.get_axis_aligned_bounding_box()

    #     visualizer([pcd, bbox])

    #     pcd = pcd.crop(bbox)

    #     visualizer(pcd)

    # final = bbox+pcd

    # o3d.io.write_point_cloud(f"final_pcd_bell_pepper_w_bb.ply", final)

    visualizer([pcd, line_set])

draw_one_bounding_box(pcd, new_pcd)

In [29]:
data

[array([0.0022259 , 0.14425428, 0.63905795]),
 array([0.25812735, 0.14425428, 0.63905795]),
 array([0.0022259 , 0.35796847, 0.63905795]),
 array([0.0022259 , 0.14425428, 0.88380245]),
 array([0.25812735, 0.35796847, 0.88380245]),
 array([0.0022259 , 0.35796847, 0.88380245]),
 array([0.25812735, 0.14425428, 0.88380245]),
 array([0.25812735, 0.35796847, 0.63905795])]